In [1]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

Para este ejercicio vamos a usar el dataset [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Es un dataset clásico que consiste en 60000 imagenes de números escritos a mano, y el objetivo es clasificar los números.

Scikit-learn tiene una función [load_digits](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html) que se puede utilizar para cargar el dataset. Sin embargo, dicha función sólo tiene 1700 observaciones. Asi que lo que vamos a hacer es cargar la version completa del dataset que está almacenada en la carpeta data.

Para ello usaremos el paquete [pickle](https://docs.python.org/3/library/pickle.html) que es una forma de guardar objetos de python al disco duro y luego poder leerlos de nuevo.

In [2]:
import pickle

with open("./data/mnist.pkl", "rb") as fname:
    mnist = pickle.load(fname)

In [3]:
mnist_data = mnist["training_images"]
mnist_target = mnist["training_labels"]

Para acelerar el ejercicio, vamos a tomar una muestra de 10000 observaciones. **Si en vuestro ordenador tarda mucho, siempre podeis reducir el tamaño mas**.

In [4]:
import numpy as np
sample_size = 10000
np.random.seed(42)
random_sample_index = np.random.randint(0, mnist_data.shape[0], sample_size)
random_sample_index

array([56422, 15795,   860, ...,  9484,  5495, 28481])

In [5]:
mnist_muestra_pixeles = mnist_data[random_sample_index]
mnist_muestra_clase = mnist_target[random_sample_index]

### Usa PCA para reducir la dimensionalidad del dataset (`mnist_muestra_pixeles`) y usa el nuevo dataset como datos de entrenamiento para un clasificador que clasifique correctamente las imagenes. El criterio de evaluacion tiene que ser el criterio F1. Hay varias formas de usar el criterio F1 para casos de multiclase (en este caso hay 10 clases, del número 0 al 9). leer la [documentación del criterio F1 puede ayudar.](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)). 

Primero tenemos que evaluar como es la distribucion de las clases objetivo:

In [6]:
pd.Series(mnist_muestra_clase).value_counts(True)

1    0.1158
7    0.1036
2    0.1022
6    0.1017
8    0.1009
9    0.0989
4    0.0977
3    0.0961
0    0.0953
5    0.0878
dtype: float64

Vemos que no hay imbalance de clases, asi que podemos usar el criterio **micro** de F1.

In [7]:
from sklearn.decomposition import PCA

Elegimos reducir la dimensionalidad pero mantener un 80% de la varianza

In [8]:
pca = PCA(0.8)

In [9]:
mnist_pca = pca.fit_transform(mnist_muestra_pixeles)

In [10]:
mnist_pca.shape

(10000, 43)

Ahora creamos el estimador, vamos a usar el clasificador KNN, aunque un clasificador de Regresión logística no sería mala idea, dado que la dimensionalidad no es muy alta.

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint as sp_randint

clf = KNeighborsClassifier()

busqueda_dist_parametros = {
    "n_neighbors": sp_randint(2,10),
    "p": sp_randint(1,3),
    "weights": ["uniform", "distance"]
}

Ahora realizamos la búsqueda aleatoria

In [12]:
from sklearn.model_selection import RandomizedSearchCV

busqueda = RandomizedSearchCV(estimator=clf,
                             param_distributions=busqueda_dist_parametros,
                             n_iter=10,
                             cv=3,
                             n_jobs=-1,
                             scoring="f1_micro")
busqueda.fit(X=mnist_pca, y=mnist_muestra_clase)

RandomizedSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=-1,
                   param_distributions={'n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001DF1F912130>,
                                        'p': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001DF1F99AE80>,
                                        'weights': ['uniform', 'distance']},
                   scoring='f1_micro')

In [13]:
busqueda.best_score_

0.9589995592240688

In [14]:
busqueda.best_params_

{'n_neighbors': 6, 'p': 2, 'weights': 'distance'}

Ya tenemos los mejores parámetros para el clasificador

In [15]:
mejores_params = {'n_neighbors': 7, 'p': 2, 'weights': 'distance'}
clusterer = KNeighborsClassifier(**mejores_params)

clusterer.fit(mnist_pca, mnist_muestra_clase)

KNeighborsClassifier(n_neighbors=7, weights='distance')